In [38]:
import os
import time
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Conv2D, Flatten, GRU, Reshape
from tensorflow.keras.optimizers import Adam
import pygame
import threading

# Initialize pygame for alarm sound
pygame.mixer.init()
alarm_sound = 'alarm.wav'  # Replace with your alarm sound file
alarm_playing = False

# Paths
data_dir = 'dataset'  # Update this to your dataset root
model_path = 'drowsiness_model.h5'
img_size = (224, 224)
batch_size = 32

# Data Augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# Load dataset
def load_data(folder):
    csv_path = os.path.join(folder, 'labels.csv')
    df = pd.read_csv(csv_path)
    df['label'] = df['drowsy']  # 1 if drowsy, 0 if awake
    img_paths = [os.path.join(folder, fname) for fname in df['filename']]
    return img_paths, df['label'].values

# Prepare dataset
train_imgs, train_labels = load_data(os.path.join(data_dir, 'train'))
valid_imgs, valid_labels = load_data(os.path.join(data_dir, 'valid'))
test_imgs, test_labels = load_data(os.path.join(data_dir, 'test'))

def process_images(img_paths, labels):
    images = [img_to_array(load_img(img, target_size=img_size)) / 255.0 for img in img_paths]
    return np.array(images), np.array(labels)

X_train, y_train = process_images(train_imgs, train_labels)
X_valid, y_valid = process_images(valid_imgs, valid_labels)
X_test, y_test = process_images(test_imgs, test_labels)

# Train model if not exists
if not os.path.exists(model_path):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    model = Sequential([
        base_model,
        Conv2D(64, (3, 3), activation='relu'),
        Flatten(),
        Reshape((1, -1)),  # Preparing for GRU input
        GRU(64, return_sequences=False),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=10, batch_size=batch_size)
    model.save(model_path)
else:
    model = load_model(model_path)

# Real-time detection
cap = cv2.VideoCapture(0)
drowsy_start_time = None
drowsy_duration = 0

# Temporal smoothing buffer (store last 5 frame predictions)
prediction_history = []

def play_alarm():
    global alarm_playing
    if not alarm_playing:
        pygame.mixer.music.load(alarm_sound)
        pygame.mixer.music.play(-1)
        alarm_playing = True

def stop_alarm():
    global alarm_playing
    if alarm_playing:
        pygame.mixer.music.stop()
        alarm_playing = False

# Function to apply temporal smoothing
def apply_temporal_smoothing(prediction, history, window_size=5):
    # Add the new prediction to the history
    history.append(prediction)
    # Limit the history size to the window size
    if len(history) > window_size:
        history.pop(0)
    # Calculate the average of the predictions in the history
    smoothed_prediction = np.mean(history)
    return smoothed_prediction

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Resize the frame and normalize
    img = cv2.resize(frame, img_size) / 255.0
    img = np.expand_dims(img, axis=0)
    
    # Make prediction using the model
    prediction = model.predict(img)[0][0]

    # Apply temporal smoothing
    smoothed_prediction = apply_temporal_smoothing(prediction, prediction_history)

    # Determine state based on smoothed prediction
    state = 'Drowsy' if smoothed_prediction > 0.5 else 'Awake'
    confidence = int(smoothed_prediction * 100 if smoothed_prediction > 0.5 else (1 - smoothed_prediction) * 100)
    label = f"{state} ({confidence}%)"

    if state == 'Drowsy':
        if drowsy_start_time is None:
            drowsy_start_time = time.time()
        elif time.time() - drowsy_start_time >= drowsy_duration:
            threading.Thread(target=play_alarm).start()
    else:
        drowsy_start_time = None
        stop_alarm()

    # Display label and color the text based on the state
    cv2.putText(frame, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, 
                (0, 0, 255) if state == 'Drowsy' else (0, 255, 0), 2)
    
    # Show the frame
    cv2.imshow('Drowsiness Detection', frame)

    # Exit condition
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        stop_alarm()
        break


ImportError: cannot import name 'OrderedDict' from 'typing' (c:\users\aryan\appdata\local\programs\python\python37\lib\typing.py)

In [35]:
import sys
print("Python version:", sys.version)

Python version: 3.7.0 (v3.7.0:1bf9cc5093, Jun 27 2018, 04:59:51) [MSC v.1914 64 bit (AMD64)]


In [36]:
!python -m pip uninstall tensorflow -y
!python -m pip install tensorflow==2.8.4

Found existing installation: tensorflow 2.13.0
Uninstalling tensorflow-2.13.0:
  Successfully uninstalled tensorflow-2.13.0
Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 438.4/438.4 MB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 1.4/1.4 MB 4.5 MB/s eta 0:00:00
   ---------------------------------------- 896.1/896.1 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 5.8/5.8 MB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 781.3/781.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorboard-data-ser

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires keras<2.14,>=2.13.1, but you have keras 2.8.0 which is incompatible.
tensorflow-intel 2.13.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.13.0 requires tensorboard<2.14,>=2.13, but you have tensorboard 2.8.0 which is incompatible.
tensorflow-intel 2.13.0 requires tensorflow-estimator<2.14,>=2.13.0, but you have tensorflow-est

In [2]:
pip install cv2

Note: you may need to restart the kernel to use updated packages.


  Could not find a version that satisfies the requirement cv2 (from versions: )
No matching distribution found for cv2
You are using pip version 10.0.1, however version 24.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
pip install --upgrade pip

  Found existing installation: pip 10.0.1
    Uninstalling pip-10.0.1:
      Successfully uninstalled pip-10.0.1
Note: you may need to restart the kernel to use updated packages.


tensorboard 2.11.2 has requirement setuptools>=41.0.0, but you'll have setuptools 39.0.1 which is incompatible.


In [6]:
# Install TensorFlow (includes Keras)
!pip install tensorflow

# Install OpenCV for computer vision
!pip install opencv-python

# Install NumPy (usually comes with most Python installations)
!pip install numpy

# Install Pandas for data manipulation
!pip install pandas

# Install Pygame for game development/multimedia
!pip install pygame

# Alternative: Install all at once
!pip install tensorflow opencv-python numpy pandas pygame

# If you're using conda instead of pip:
# !conda install tensorflow opencv numpy pandas
# !conda install -c conda-forge pygame

  ERROR: Could not find a version that satisfies the requirement tensorflow-intel==2.13.1; platform_system == "Windows" (from tensorflow) (from versions: 0.0.1, 2.10.0.dev20220728, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0)
ERROR: No matching distribution found for tensorflow-intel==2.13.1; platform_system == "Windows" (from tensorflow)
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/17/06/68c27a523103dad5837dc5b87e71285280c4f098c60e4fe8a8db6486ab09/opencv-python-4.11.0.86.tar.gz
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'


ERROR: Exception:
Traceback (most recent call last):
  File "c:\program files\python38\lib\site-packages\pip\_internal\cli\base_command.py", line 188, in main
    status = self.run(options, args)
  File "c:\program files\python38\lib\site-packages\pip\_internal\commands\install.py", line 345, in run
    resolver.resolve(requirement_set)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 196, in resolve
    self._resolve_one(requirement_set, req)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 359, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 305, in _get_abstract_dist_for
    abstract_dist = self.preparer.prepare_linked_requirement(
  File "c:\program files\python38\lib\site-packages\pip\_internal\operations\prepare.py", line 214, in prepare_linked_requirement
    abstract_dist.pr

  Using cached https://files.pythonhosted.org/packages/69/65/0d47953afa0ad569d12de5f65d964321c208492064c38fe3b0b9744f8d44/numpy-1.24.4-cp38-cp38-win_amd64.whl


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\program files\\python38\\Lib\\site-packages\\numpy'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/ec/57/56b9bcc3c9c6a792fcbaf139543cee77261f3651ca9da0c93f5c1221264b/python_dateutil-2.9.0.post0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/69/65/0d47953afa0ad569d12de5f65d964321c208492064c38fe3b0b9744f8d44/numpy-1.24.4-cp38-cp38-win_amd64.whl


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\program files\\python38\\Lib\\site-packages\\tzdata'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\program files\\python38\\Lib\\site-packages\\pygame'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


  Using cached https://files.pythonhosted.org/packages/fb/59/3eb58629e3749d9f4fc1e522487af369f9bd4c451f465d3054961fab6bf8/tensorflow-2.13.1-cp38-cp38-win_amd64.whl
  Using cached https://files.pythonhosted.org/packages/17/06/68c27a523103dad5837dc5b87e71285280c4f098c60e4fe8a8db6486ab09/opencv-python-4.11.0.86.tar.gz
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'


ERROR: Exception:
Traceback (most recent call last):
  File "c:\program files\python38\lib\site-packages\pip\_internal\cli\base_command.py", line 188, in main
    status = self.run(options, args)
  File "c:\program files\python38\lib\site-packages\pip\_internal\commands\install.py", line 345, in run
    resolver.resolve(requirement_set)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 196, in resolve
    self._resolve_one(requirement_set, req)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 359, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 305, in _get_abstract_dist_for
    abstract_dist = self.preparer.prepare_linked_requirement(
  File "c:\program files\python38\lib\site-packages\pip\_internal\operations\prepare.py", line 214, in prepare_linked_requirement
    abstract_dist.pr

In [9]:
!pip install opencv-python

  Using cached https://files.pythonhosted.org/packages/17/06/68c27a523103dad5837dc5b87e71285280c4f098c60e4fe8a8db6486ab09/opencv-python-4.11.0.86.tar.gz
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'


ERROR: Exception:
Traceback (most recent call last):
  File "c:\program files\python38\lib\site-packages\pip\_internal\cli\base_command.py", line 188, in main
    status = self.run(options, args)
  File "c:\program files\python38\lib\site-packages\pip\_internal\commands\install.py", line 345, in run
    resolver.resolve(requirement_set)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 196, in resolve
    self._resolve_one(requirement_set, req)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 359, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 305, in _get_abstract_dist_for
    abstract_dist = self.preparer.prepare_linked_requirement(
  File "c:\program files\python38\lib\site-packages\pip\_internal\operations\prepare.py", line 214, in prepare_linked_requirement
    abstract_dist.pr

In [11]:
!pip install opencv-contrib-python

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'


ERROR: Exception:
Traceback (most recent call last):
  File "c:\program files\python38\lib\site-packages\pip\_internal\cli\base_command.py", line 188, in main
    status = self.run(options, args)
  File "c:\program files\python38\lib\site-packages\pip\_internal\commands\install.py", line 345, in run
    resolver.resolve(requirement_set)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 196, in resolve
    self._resolve_one(requirement_set, req)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 359, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)
  File "c:\program files\python38\lib\site-packages\pip\_internal\legacy_resolve.py", line 305, in _get_abstract_dist_for
    abstract_dist = self.preparer.prepare_linked_requirement(
  File "c:\program files\python38\lib\site-packages\pip\_internal\operations\prepare.py", line 214, in prepare_linked_requirement
    abstract_dist.pr

In [13]:
!python -m pip install --upgrade pip

  Using cached https://files.pythonhosted.org/packages/c9/bc/b7db44f5f39f9d0494071bddae6880eb645970366d0a200022a1a93d57f5/pip-25.0.1-py3-none-any.whl
  Found existing installation: pip 19.2.3
    Uninstalling pip-19.2.3:


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'c:\\program files\\python38\\lib\\site-packages\\pip-19.2.3.dist-info\\entry_points.txt'
Consider using the `--user` option or check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [21]:
!python -m pip install --upgrade pip --user
!pip install opencv-python --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [17]:
!python -m pip install tensorflow --user
!python -m pip install pandas --user
!python -m pip install pygame --user

INFO: pip is looking at multiple versions of tensorflow to determine which version is compatible with other requirements. This could take a while.
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 276.5/276.5 MB 4.1 MB/s eta 0:00:00
Using cached astunparse-1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
   ---------------------------------------- 10.8/10.8 MB 5.5 MB/s eta 0:00:00
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
   ---------------------------------------- 10.6/10.6 MB 3.2 MB/s eta 0:00:00


In [33]:
import cv2
import numpy as np
import pandas as pd
print("All imports successful!")
print("OpenCV version:", cv2.__version__)

All imports successful!
OpenCV version: 4.11.0


In [23]:
import sys
print("Python executable:", sys.executable)
print("Python paths:")
for path in sys.path:
    print(f"  {path}")

Python executable: c:\users\aryan\appdata\local\programs\python\python37\python.exe
Python paths:
  C:\Local Disk E_102620232316\Engineering\Fourth Year\BE Project 24-25\DDD02
  c:\users\aryan\appdata\local\programs\python\python37\python37.zip
  c:\users\aryan\appdata\local\programs\python\python37\DLLs
  c:\users\aryan\appdata\local\programs\python\python37\lib
  c:\users\aryan\appdata\local\programs\python\python37
  
  c:\users\aryan\appdata\local\programs\python\python37\lib\site-packages
  c:\users\aryan\appdata\local\programs\python\python37\lib\site-packages\win32
  c:\users\aryan\appdata\local\programs\python\python37\lib\site-packages\win32\lib
  c:\users\aryan\appdata\local\programs\python\python37\lib\site-packages\Pythonwin
  c:\users\aryan\appdata\local\programs\python\python37\lib\site-packages\IPython\extensions
  C:\Users\ARYAN\.ipython


In [24]:
import site
user_site = site.getusersitepackages()
print(f"\nUser site-packages: {user_site}")

if user_site not in sys.path:
    sys.path.append(user_site)
    print("Added user site-packages to Python path")


User site-packages: C:\Users\ARYAN\AppData\Roaming\Python\Python37\site-packages
Added user site-packages to Python path


In [31]:
try:
    import cv2
    print(f"✓ OpenCV imported successfully! Version: {cv2.__version__}")
except ImportError as e:
    print(f"✗ OpenCV still not found: {e}")

✓ OpenCV imported successfully! Version: 4.11.0


In [32]:
import subprocess
result = subprocess.run([sys.executable, '-m', 'pip', 'list'], 
                       capture_output=True, text=True)
if 'opencv-python' in result.stdout:
    print("✓ OpenCV is installed")
    print("Found opencv-python in pip list")
else:
    print("✗ OpenCV not found in pip list")

✓ OpenCV is installed
Found opencv-python in pip list


In [28]:
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])

In [30]:
# Uncomment and run if opencv still not working:
install_package('opencv-python')